# Práctica Métodos de Simulación - Parte 3

## Elena Rivas, Teresa Grau, Ignacio Casso

## Enunciado

**El problema de las n reinas.**

El problema de las n-reinas consiste en colocar n reinas en un tablero de ajedrez de
n x n de tal manera que no sea posible que dos reinas se capturen entre si, es decir,
que no estén en la misma fila, ni en la misma columna ni en la misma diagonal. Se
dice que hay una colisión si hay dos reinas que se pueden capturar entre si.

Se trata pues de encontrar una configuración (elegir las n celdas donde colocar a las
reinas) que minimice el número total de colisiones. Obtener la solución óptima para
n=9.

Mostrar gráficamente la evolución del valor de la función fitness y del parámetro
temperatura considerados a lo largo de las iteraciones de la metaheurı́stica.

**Enfriamiento simulado**

El Enfriamiento simulado es una técnica de optimización combinatorial que se usa para afrontar problemas de gran complejidad matemática, de modo que se obtengan soluciones cercanas a la óptima. 
Se fundamenta en el proceso físico de calentamiento de un sólido, seguido por un enfriamiento hasta lograr un estado cristalino con una estructura perfecta. Durante este proceso, la energía libre del sólido es minimizada. 

"A una temperatura T, en el estado con nivel de energía Eì, se genera el estado siguiente j a través de de una pequeña perturbación. Si la diferencia de energía Ej – Ei es menor o igual a cero, el estado j es aceptado. Si la diferencia de energía es mayor que cero, el estado j es aceptado con una probabilidad P, donde KB es la constante de Boltzmann. Si la disminución de la temperatura es hecha de manera paulatina, el sólido puede alcanzar el estado de equilibrio en cada nivel de temperatura." 

## Planteamiento

#### El problema de las n reinas se puede plantear de dos maneras: 

- A) ubicando n reinas de manera que se minimicen las colisiones 
- B) maximizar el número de reinas en el tablero sujeto a la restricción de que no existan colisiones

## Soluciones

En esta práctica nos vamos a centrar en el planteamiento decrito anteriormente como A), para el que queremos minimizar el número de colisiones entre las reinas. Para éste proponemos dos implementaciones distintas para el algoritmo de enfriamiento simulado, que se diferencian en el espacio de búsqueda y en la codificación de las soluciones:
- A1)  En esta solución consideramos todo el espacio de búsqueda, es decir, todas las posibles configuraciones del tablero con $n$ reinas en él. 
- A2) En este caso tenemos en cuenta que por necesidad debe haber una reina en cada fila y otra en cada columna, y codificamos las soluciones como permutaciones de $n$ elementos, reduciendo el espacio de soluciones de un cardinal de $(n^2)^n$ a uno de $n!$. 

Para cada una de ellas se exploran distintas elecciones de parámetros y los resultados que se obtienen.

## Caso A1) Consideramos todas las posibles configuraciones del tablero con n reinas.

### Codificación de las soluciones

Codificaremos las soluciones como listas de 9 posiciones, siendo cada posición una tupla de 2 elementos. Para evitar redundancia en la codificación, se impone que esa lista esté ordenada (lexicográficamente), y para evitar soluciones no factibles, se prohiben los elementos repetidos en una lista.

In [26]:
from random import random, randint

### Función objetivo

La función objetivo a minimizar será el número de colisiones entre reinas.

In [27]:
def num_collisions(positions):
    
    if positions==[]:
        return 0
    else:
        queen, positions2 = positions[0], positions[1:]
        num_colls = 0
        for queen2 in positions2:
            if collision(queen,queen2):
                num_colls += 1
        return num_colls + num_collisions(positions2)
    
def collision(queen1,queen2):
    i1,j1 = queen1
    i2,j2 = queen2
    
    if i1==i2 or j1==j2 or abs(i1-i2)==abs(j1-j2):
        return True
    else:
        return False

In [28]:
def numCollisionsFitness(sol,n):
    return num_collisions(sol)

### Entorno de un punto en el espacio de búsqueda

Proponemos los siguientes entornos para una solución:

La primera, las soluciones obtenidas al mover una reina una posición en cualquier dirección.

In [43]:
moves = [(0,1),(0,-1),(1,0),(-1,0),(1,1),(1,-1),(-1,1),(-1,-1)]

def oneMoveRandomNeighbor(sol,n):
    queens = sol[:] # copy
    i = randint(0,n-1)
    j = randint(0,7)
    qx,qy = queens[i]
    mx,my = moves[j]
    nqx,nqy = qx+mx, qy+my
    if 0<nqx and 0 < nqy and n >= nqx and n>=nqy:
        new_queen = (nqx,nqy)
    else:
        new_queen = qx,qy
    queens[i] = new_queen
    return queens

En esta solución no se restringe el problema de que un determinado movimiento nos saque del tablero o que de que haya 2 reinas en la misma posición.

### Valor inicial de la temperatura

La temperatura debe ser tal en el proceso que permita una adecuada exploración en las diferentes etapas del enfriamiento simulado. Existen diferentes formas para calcular la temperatura inicial, siendo una de ellas simular el proceso para la primera cadena de Markov.  

Para este caso vamos a proponer los siguientes valores iniciales de temperatura:

In [30]:
def standardInitialTemp():
    return 0.9

### Variación de la temperatura

Proponemos las siguientes actualizaciones de temperatura:

En primer lugar, la propuesta en las transparencias, con $L=10,~ \alpha = 0.95$

In [31]:
def updateTempStandard(T,i):
    if i%10 == 0:
        newT = 0.95*T
    else:
        newT = T
    return newT

### Probabilidad de aceptación de nuevas soluciones

En este caso proponemos la estándar:

**Duda:** No debería estar la probabilidad entre 0 y 1?

In [45]:
from math import exp

In [46]:
def prob_aceptacion_standard(f1,f,T):
    x = -(f1-f)/T
    return exp(x)

### Criterio de parada

Se pueden usar varios criterios de parada:
    
- Parar si se ha encontrado la solución óptima     
- Fijar un número determinado de niveles de temperatura, usualmente entre 6 y 50. 
- Terminar el procesos si la función objetivo no mejora para varios niveles consecutivos. 
- Parar si no se cumple con un número mínimo de aceptaciones en el nivel de temperatura.

El caso que vamos a utilizar es en el que la parada se produce cuando se encuentra la solución óptima:

In [47]:
def solutionFound(f,_):
    return f==0

### Configuración inicial

De la configuración inicial depende la rapidez para encontrar una solución. 
Por ejemplo para una inicialización hecha de forma aleatoria, se tiene que alrededor de la mitad de las reinas están en colisión. Existen estrategias para inicializar la configuración de tal manera que se reduzca el número de colisiones al comienzo del proceso. 

La solución que planteamos es una de las más simples a través de la que se colocan todas las reinas en la primera fila.

In [48]:
def initialSolSameRow(n):
    return [(1,i+1) for i in range(n)]

In [36]:
initialSols=[initialSolSameRow]

### Algoritmo

In [89]:
from random import random

In [62]:
def queens(n):
    sol = initialSol(n)
    f = fitness(sol,n)
    best_sol = sol
    best_f = f
    T = initialTemp()
    fitness_values=[f]
    temp=[T]
    i=1    
    while not criterio_parada(best_f,i):
        new_sol = random_neighbor(sol,n)
        new_f = fitness(new_sol,n)
        if new_f < f:
            sol = new_sol
            f = new_f
            if f < best_f:
                best_sol=sol
                best_f = f
        else:
            u = random()
            p = prob_aceptacion(new_f,f,T)
            if p > u:
                sol = new_sol
                f = new_f
        update_temp(T,i)
        i += 1
    return sol

In [110]:
def queens(n):
    sol = initialSol(n)
    f = fitness(sol,n)
    best_sol = sol
    best_f = f
    T = initialTemp()
    fitness_values=[f]
    temp=[T]    
    i=1    
    while not criterio_parada(best_f,i):
        new_sol = random_neighbor(sol,n)
        new_f = fitness(new_sol,n)
        if new_f < f:
            sol = new_sol
            f = new_f
            if f < best_f:
                best_sol=sol
                best_f = f
        else:
            u = random()
            p = prob_aceptacion(new_f,f,T)
            if p > u:
                sol = new_sol
                f = new_f         
        T = update_temp(T,i)  
        print i,T,f
        i += 1        
    return sol
  

### Resultados

Para probar la implementación del algoritmo se ha resuelto el problema de las N reinas para distintos valores de N. Para cada tablero se ha lanzado 5 veces el programa. 

In [111]:
initialSol=initialSolSameRow
fitness = numCollisionsFitness
initialTemp = standardInitialTemp
criterio_parada = solutionFound
random_neighbor = oneMoveRandomNeighbor
update_temp = updateTempStandard
prob_aceptacion = prob_aceptacion_standard

In [112]:
queens(9)

1 0.9 36
2 0.9 36
3 0.9 30
4 0.9 30
5 0.9 26
6 0.9 23
7 0.9 22
8 0.9 22
9 0.9 19
10 0.855 19
11 0.855 19
12 0.855 19
13 0.855 19
14 0.855 19
15 0.855 19
16 0.855 17
17 0.855 15
18 0.855 16
19 0.855 16
20 0.81225 16
21 0.81225 16
22 0.81225 16
23 0.81225 15
24 0.81225 15
25 0.81225 15
26 0.81225 15
27 0.81225 15
28 0.81225 15
29 0.81225 15
30 0.7716375 15
31 0.7716375 15
32 0.7716375 15
33 0.7716375 15
34 0.7716375 14
35 0.7716375 14
36 0.7716375 14
37 0.7716375 14
38 0.7716375 14
39 0.7716375 13
40 0.733055625 13
41 0.733055625 13
42 0.733055625 13
43 0.733055625 12
44 0.733055625 12
45 0.733055625 11
46 0.733055625 11
47 0.733055625 9
48 0.733055625 9
49 0.733055625 9
50 0.69640284375 8
51 0.69640284375 8
52 0.69640284375 8
53 0.69640284375 8
54 0.69640284375 8
55 0.69640284375 8
56 0.69640284375 8
57 0.69640284375 8
58 0.69640284375 8
59 0.69640284375 8
60 0.661582701562 8
61 0.661582701562 8
62 0.661582701562 8
63 0.661582701562 8
64 0.661582701562 8
65 0.661582701562 8
66 0.6615827

1088 0.00353502004393 3
1089 0.00353502004393 3
1090 0.00335826904174 3
1091 0.00335826904174 3
1092 0.00335826904174 3
1093 0.00335826904174 3
1094 0.00335826904174 3
1095 0.00335826904174 3
1096 0.00335826904174 3
1097 0.00335826904174 3
1098 0.00335826904174 3
1099 0.00335826904174 3
1100 0.00319035558965 3
1101 0.00319035558965 3
1102 0.00319035558965 3
1103 0.00319035558965 3
1104 0.00319035558965 3
1105 0.00319035558965 3
1106 0.00319035558965 3
1107 0.00319035558965 3
1108 0.00319035558965 3
1109 0.00319035558965 3
1110 0.00303083781017 3
1111 0.00303083781017 3
1112 0.00303083781017 3
1113 0.00303083781017 3
1114 0.00303083781017 3
1115 0.00303083781017 3
1116 0.00303083781017 3
1117 0.00303083781017 3
1118 0.00303083781017 3
1119 0.00303083781017 3
1120 0.00287929591966 3
1121 0.00287929591966 3
1122 0.00287929591966 3
1123 0.00287929591966 3
1124 0.00287929591966 3
1125 0.00287929591966 3
1126 0.00287929591966 3
1127 0.00287929591966 3
1128 0.00287929591966 3
1129 0.002879295

1763 0.000108043100919 3
1764 0.000108043100919 3
1765 0.000108043100919 3
1766 0.000108043100919 3
1767 0.000108043100919 3
1768 0.000108043100919 3
1769 0.000108043100919 3
1770 0.000102640945873 3
1771 0.000102640945873 3
1772 0.000102640945873 3
1773 0.000102640945873 3
1774 0.000102640945873 3
1775 0.000102640945873 3
1776 0.000102640945873 3
1777 0.000102640945873 3
1778 0.000102640945873 3
1779 0.000102640945873 3
1780 9.75088985796e-05 3
1781 9.75088985796e-05 3
1782 9.75088985796e-05 3
1783 9.75088985796e-05 3
1784 9.75088985796e-05 3
1785 9.75088985796e-05 3
1786 9.75088985796e-05 3
1787 9.75088985796e-05 3
1788 9.75088985796e-05 3
1789 9.75088985796e-05 3
1790 9.26334536506e-05 3
1791 9.26334536506e-05 3
1792 9.26334536506e-05 3
1793 9.26334536506e-05 3
1794 9.26334536506e-05 3
1795 9.26334536506e-05 3
1796 9.26334536506e-05 3
1797 9.26334536506e-05 3
1798 9.26334536506e-05 3
1799 9.26334536506e-05 3
1800 8.80017809681e-05 3
1801 8.80017809681e-05 3
1802 8.80017809681e-05 3


2481 2.68965527325e-06 3
2482 2.68965527325e-06 3
2483 2.68965527325e-06 3
2484 2.68965527325e-06 3
2485 2.68965527325e-06 3
2486 2.68965527325e-06 3
2487 2.68965527325e-06 3
2488 2.68965527325e-06 3
2489 2.68965527325e-06 3
2490 2.55517250959e-06 3
2491 2.55517250959e-06 3
2492 2.55517250959e-06 3
2493 2.55517250959e-06 3
2494 2.55517250959e-06 3
2495 2.55517250959e-06 3
2496 2.55517250959e-06 3
2497 2.55517250959e-06 3
2498 2.55517250959e-06 3
2499 2.55517250959e-06 3
2500 2.42741388411e-06 3
2501 2.42741388411e-06 3
2502 2.42741388411e-06 3
2503 2.42741388411e-06 3
2504 2.42741388411e-06 3
2505 2.42741388411e-06 3
2506 2.42741388411e-06 3
2507 2.42741388411e-06 3
2508 2.42741388411e-06 3
2509 2.42741388411e-06 3
2510 2.3060431899e-06 3
2511 2.3060431899e-06 3
2512 2.3060431899e-06 3
2513 2.3060431899e-06 3
2514 2.3060431899e-06 3
2515 2.3060431899e-06 3
2516 2.3060431899e-06 3
2517 2.3060431899e-06 3
2518 2.3060431899e-06 3
2519 2.3060431899e-06 3
2520 2.19074103041e-06 3
2521 2.190

3097 1.17716283894e-07 3
3098 1.17716283894e-07 3
3099 1.17716283894e-07 3
3100 1.11830469699e-07 3
3101 1.11830469699e-07 3
3102 1.11830469699e-07 3
3103 1.11830469699e-07 3
3104 1.11830469699e-07 3
3105 1.11830469699e-07 3
3106 1.11830469699e-07 3
3107 1.11830469699e-07 3
3108 1.11830469699e-07 3
3109 1.11830469699e-07 3
3110 1.06238946214e-07 3
3111 1.06238946214e-07 3
3112 1.06238946214e-07 3
3113 1.06238946214e-07 3
3114 1.06238946214e-07 3
3115 1.06238946214e-07 3
3116 1.06238946214e-07 3
3117 1.06238946214e-07 3
3118 1.06238946214e-07 3
3119 1.06238946214e-07 3
3120 1.00926998903e-07 3
3121 1.00926998903e-07 3
3122 1.00926998903e-07 3
3123 1.00926998903e-07 3
3124 1.00926998903e-07 3
3125 1.00926998903e-07 3
3126 1.00926998903e-07 3
3127 1.00926998903e-07 3
3128 1.00926998903e-07 3
3129 1.00926998903e-07 3
3130 9.58806489582e-08 3
3131 9.58806489582e-08 3
3132 9.58806489582e-08 3
3133 9.58806489582e-08 3
3134 9.58806489582e-08 3
3135 9.58806489582e-08 3
3136 9.58806489582e-08 3


...

25132 9.41158134683e-57 3
25133 9.41158134683e-57 3
25134 9.41158134683e-57 3
25135 9.41158134683e-57 3
25136 9.41158134683e-57 3
25137 9.41158134683e-57 3
25138 9.41158134683e-57 3
25139 9.41158134683e-57 3
25140 8.94100227949e-57 3
25141 8.94100227949e-57 3
25142 8.94100227949e-57 3
25143 8.94100227949e-57 3
25144 8.94100227949e-57 3
25145 8.94100227949e-57 3
25146 8.94100227949e-57 3
25147 8.94100227949e-57 3
25148 8.94100227949e-57 3
25149 8.94100227949e-57 3
25150 8.49395216551e-57 3
25151 8.49395216551e-57 3
25152 8.49395216551e-57 3
25153 8.49395216551e-57 3
25154 8.49395216551e-57 3
25155 8.49395216551e-57 3
25156 8.49395216551e-57 3
25157 8.49395216551e-57 3
25158 8.49395216551e-57 3
25159 8.49395216551e-57 3
25160 8.06925455724e-57 3
25161 8.06925455724e-57 3
25162 8.06925455724e-57 3
25163 8.06925455724e-57 3
25164 8.06925455724e-57 3
25165 8.06925455724e-57 3
25166 8.06925455724e-57 3
25167 8.06925455724e-57 3
25168 8.06925455724e-57 3
25169 8.06925455724e-57 3
25170 7.6657

25781 3.35503414222e-58 3
25782 3.35503414222e-58 3
25783 3.35503414222e-58 3
25784 3.35503414222e-58 3
25785 3.35503414222e-58 3
25786 3.35503414222e-58 3
25787 3.35503414222e-58 3
25788 3.35503414222e-58 3
25789 3.35503414222e-58 3
25790 3.18728243511e-58 3
25791 3.18728243511e-58 3
25792 3.18728243511e-58 3
25793 3.18728243511e-58 3
25794 3.18728243511e-58 3
25795 3.18728243511e-58 3
25796 3.18728243511e-58 3
25797 3.18728243511e-58 3
25798 3.18728243511e-58 3
25799 3.18728243511e-58 3
25800 3.02791831336e-58 3
25801 3.02791831336e-58 3
25802 3.02791831336e-58 3
25803 3.02791831336e-58 3
25804 3.02791831336e-58 3
25805 3.02791831336e-58 3
25806 3.02791831336e-58 3
25807 3.02791831336e-58 3
25808 3.02791831336e-58 3
25809 3.02791831336e-58 3
25810 2.87652239769e-58 3
25811 2.87652239769e-58 3
25812 2.87652239769e-58 3
25813 2.87652239769e-58 3
25814 2.87652239769e-58 3
25815 2.87652239769e-58 3
25816 2.87652239769e-58 3
25817 2.87652239769e-58 3
25818 2.87652239769e-58 3
25819 2.8765

26385 1.54565748527e-59 3
26386 1.54565748527e-59 3
26387 1.54565748527e-59 3
26388 1.54565748527e-59 3
26389 1.54565748527e-59 3
26390 1.468374611e-59 3
26391 1.468374611e-59 3
26392 1.468374611e-59 3
26393 1.468374611e-59 3
26394 1.468374611e-59 3
26395 1.468374611e-59 3
26396 1.468374611e-59 3
26397 1.468374611e-59 3
26398 1.468374611e-59 3
26399 1.468374611e-59 3
26400 1.39495588045e-59 3
26401 1.39495588045e-59 3
26402 1.39495588045e-59 3
26403 1.39495588045e-59 3
26404 1.39495588045e-59 3
26405 1.39495588045e-59 3
26406 1.39495588045e-59 3
26407 1.39495588045e-59 3
26408 1.39495588045e-59 3
26409 1.39495588045e-59 3
26410 1.32520808643e-59 3
26411 1.32520808643e-59 3
26412 1.32520808643e-59 3
26413 1.32520808643e-59 3
26414 1.32520808643e-59 3
26415 1.32520808643e-59 3
26416 1.32520808643e-59 3
26417 1.32520808643e-59 3
26418 1.32520808643e-59 3
26419 1.32520808643e-59 3
26420 1.25894768211e-59 3
26421 1.25894768211e-59 3
26422 1.25894768211e-59 3
26423 1.25894768211e-59 3
26424 

27054 4.97272927189e-61 3
27055 4.97272927189e-61 3
27056 4.97272927189e-61 3
27057 4.97272927189e-61 3
27058 4.97272927189e-61 3
27059 4.97272927189e-61 3
27060 4.72409280829e-61 3
27061 4.72409280829e-61 3
27062 4.72409280829e-61 3
27063 4.72409280829e-61 3
27064 4.72409280829e-61 3
27065 4.72409280829e-61 3
27066 4.72409280829e-61 3
27067 4.72409280829e-61 3
27068 4.72409280829e-61 3
27069 4.72409280829e-61 3
27070 4.48788816788e-61 3
27071 4.48788816788e-61 3
27072 4.48788816788e-61 3
27073 4.48788816788e-61 3
27074 4.48788816788e-61 3
27075 4.48788816788e-61 3
27076 4.48788816788e-61 3
27077 4.48788816788e-61 3
27078 4.48788816788e-61 3
27079 4.48788816788e-61 3
27080 4.26349375949e-61 3
27081 4.26349375949e-61 3
27082 4.26349375949e-61 3
27083 4.26349375949e-61 3
27084 4.26349375949e-61 3
27085 4.26349375949e-61 3
27086 4.26349375949e-61 3
27087 4.26349375949e-61 3
27088 4.26349375949e-61 3
27089 4.26349375949e-61 3
27090 4.05031907151e-61 3
27091 4.05031907151e-61 3
27092 4.0503

KeyboardInterrupt: 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Se muestra en el gráfico siguiente la evolución del valor de la función fitness a lo largo de las iteraciones.

In [ ]:
plt.plot(i, f)
plt.show()

En el siguiente gráfico se muestra la evolución del parámetro temperatura en las diferentes iteraciones.

In [ ]:
plt.plot(i, T)
plt.show()

Se muestra en la tabla siguiente el promedio del número de iteraciones en las que se alcanzó la solución del problema.
 

En la siguiente tabla aparecen los datos para las 10 ejecuciones con diferentes valores de N.
Se observa que el número de iteraciones crece de forma lineal con el tamaño del tablero. 
El tiempo de cómputo a su vez es proporcional al número de iteraciones.
El tiempo para iteración se mantiene constante al aumentar la dimensión del tablero, debido a que se analizan 8 diagonales en cada iteración sin importar el tamaño del problema. 

## Caso A2) Tenemos en cuenta que debe haber una reina en cada fila y otra en cada columna, y reducimos el problema a una permutación.

Puesto que dos reinas colisionan cuando están en la misma columna, fila o diagonal esto significa que el problema a solucionar pasar por evitar que las reinas vayan en una misma fila, es decir, que a cada reina le corresponda una fila de manera que la reina k se sitúa en la fila k. Y que del mismo modo la columna en la que se coloca tendrá que ser también diferente para cada reina. De este modo el problema se convierte en un problema de permutación. 
 
Si R(i) es el índice de la columna donde se localiza la reina i, entonces la configuración del tablero se describe con el vector R. Para el caso del tablero 9x9 del vector de configuración R es [1, 3, 5, 7, 9, 2, 4, 6, 8]. 

Por tanto si a cada reina se le asigna una fila y una columna que no comparte con otra, no existen colisiones entre reinas por filas o columnas, entonces el problema que debe analizarse es principalmente la configuración para las diagonales. Así si dos o más reinas comparten el mismo índice para una diagonal, entonces existe una colisión.

La configuración inicial es creada según ... y la temperatura inicial se calcula ... . 
Se genera una nueva configuración a través del intercambio de columnas entre dos reinas escogidas de forma aleatoria, pero dándole mayor probabilidad a las reinas que están en colisión de acuerdo con una lista que se va actualizando. 